# Basic objects

> Basic object types and methods utilized in `pybx`

In [ ]:
#| default_exp basics

In [ ]:
#| export
import warnings
import inspect
from typing import Union
from numpy.typing import ArrayLike

import numpy as np
from fastcore.dispatch import explode_types
from fastcore.foundation import L
from fastcore.basics import concat, store_attr, patch, GetAttr
from fastcore.xtras import is_listy

from pybx.ops import mul, sub, intersection_box, make_single_iterable, voc_keys, update_keys
from pybx.excepts import *

COORD_TYPES = (np.float_, np.int_, int)
ITER_TYPES = (np.ndarray, list, L)
ITER_TYPES_TUPLE = (tuple, )
ITER_TYPES_EXTRA = (dict, )
ALL_TYPES = COORD_TYPES + ITER_TYPES

In [ ]:
import json
from fastcore.test import test_eq, test_fail, test_warns, ExceptionExpected

Anchor box coordinates of type `list`/`dict`/`json`/`array` can be converted 
to a `Bx` instance. Once wrapped as a `Bx` instance, some interesting properties can
be calculated from the coordinates. 

In [ ]:
# | export
class Bx:
    """Interface for all future Bx's"""

    def __init__(self, coords, label: list = None):
        label = label if label else []
        label = L(label) if not is_listy(label) else label
        coords = [coords] if len(coords) > 1 else coords  # make list of list
        # other props
        _coords = coords[0]  # internatl representation as a list
        assert len(_coords) == 4, f"Expected 4 items in _coords, got {_coords}"
        x_min, y_min, x_max, y_max = _coords
        store_attr("x_min, y_min, x_max, y_max, _coords, coords, label")

    def __str__(self):
        return f"Bx(coords={self.coords}, label={self.label})"

    def __repr__(self):
        return self.__str__()

    def __len__(self):
        return len(self.coords)

    def get_coords(self):
        return self.coords

    def get_label(self):
        return self.label

    @property
    def w(self):
        """Calculate width"""
        return sub(*self._coords[::2][::-1])

    @property
    def h(self):
        """Calculate height"""
        return sub(*self._coords[1::2][::-1])

    @property
    def cx(self):
        """Calculate centroid-x"""
        return self.x_min + self.w / 2

    @property
    def cy(self):
        """Calculate centroid-y"""
        return self.y_min + self.h / 2

    @property
    def area(self):
        """Calculates the absolute value of the area of the box."""
        return abs(mul(self.w, self.h))

    @property
    def values(self):
        """Returns the coordinates and label as a single list."""
        return L([[*self._coords, *self.label]])

    @property
    def valid(self):
        """Checks for validity of the box and returns a boolean.
        From `v0.1.3`, validity implies that the box has non-zero area.
        """
        v_area = bool(self.area)  # False if 0
        v_all = np.array([v_area])
        return True if v_all.all() else False

    @property
    def xywh(self):
        """Converts the `pascal_voc` bounding box to `coco` format."""
        return L([[self.x_min, self.y_min, self.w, self.h]])

    def yolo(self, w=None, h=None, normalize=False):
        """Converts the `pascal_voc` bounding box to `yolo` centroids format.
        :param normalize: Whether to normalize the bounding box with image width and height.
        :param w: Width of image. Not to be confused with `BaseBx` attribute `w`.
        :param h: Height of image. Not to be confused with `BaseBx` attribute `h`.
        """
        if normalize:
            assert (
                w is not None
            ), f"{inspect.stack()[0][3]} of {__name__}: Expected width and height of image with normalize={normalize}."
            assert (
                h is not None
            ), f"{inspect.stack()[0][3]} of {__name__}: Expected width and height of image with normalize={normalize}."
            return L([[self.cx / w, self.cy / h, self.w / w, self.h / h]])
        return L([[self.cx, self.cy, self.w, self.h]])


Initializing an empty `Bx` class. It does a whole lot of things!

Generate random coordinates for one anchor boxes.

In [ ]:
np.random.seed(42)
annots = [sorted([np.random.randint(100) for i in range(4)])]
annots

[[14, 51, 71, 92]]

If a single list is passed, `Bx` will make it a list of list.

In [ ]:
Bx(annots[0])

Bx(coords=[[14, 51, 71, 92]], label=[])

So the correct way to do it would be to pass a list of list.

In [ ]:
annots

[[14, 51, 71, 92]]

In [ ]:
b = Bx(annots)
b

Bx(coords=[[14, 51, 71, 92]], label=[])

In [ ]:
len(b)

1

In [ ]:
b.cx

42.5

In [ ]:
b.yolo()

(#1) [[42.5, 71.5, 57, 41]]

To get normalized coordinates wrt to the image dimensions.

In [ ]:
b.yolo(224, 224, normalize=True)

(#1) [[0.18973214285714285, 0.31919642857142855, 0.2544642857142857, 0.18303571428571427]]

In [ ]:
b.values

(#1) [[14, 51, 71, 92]]

`Bx` is inherited by all other types in `pybx`: `BaseBx`, `MultiBx`, `ListBx`, `JsonBx`, exposing the same properties.

`BaseBx` works with other types of coordinates too. 
It accepts the coordinates and label for one anchor box in a `list` or `ndarray` 
format.

In [ ]:
# | export 
class BaseBx(Bx):
    """BaseBx is the most primitive form of representing a bounding box.
    Coordinates and label of a bounding box can be wrapped as a BaseBx using:
    `bbx(coords, label)`.

    :param coords: can be of type `list` or `array` representing a single box.
        - `list` can be formatted with `label`: `[x_min, y_min, x_max, y_max, label]`
            or without `label`: `[x_min, y_min, x_max, y_max]`
        - `array` should be a 1-dimensional array of shape `(4,)`

    :param label: a `list` or `str` that has the class name or label for the object
    in the corresponding box.
    """

    def __init__(self, coords, label: list = None):
        self.index = 0  # Fixes #2, calls itself everytime
        assert isinstance(coords, (list, L, np.ndarray)), f"{__name__}: Expected type list or np.ndarray for coords, got {type(coords)}"
        assert isinstance(coords[0], ALL_TYPES), (
            f"{__name__}: Expected float, int or single-nested list or np.ndarray at coords[0], "
            f"got {type(coords[0])} with {coords[0]}"
        ) 
        super().__init__(coords, label) 
    
    def __str__(self):
        return f'BaseBx(coords={self.coords}, label={self.label})'
        

Works with arrays and lists:

In [ ]:
BaseBx(annots)

BaseBx(coords=[[14, 51, 71, 92]], label=[])

In [ ]:
b = BaseBx(annots, 'flower')

In [ ]:
b

BaseBx(coords=[[14, 51, 71, 92]], label=['flower'])

In [ ]:
b.coords

[[14, 51, 71, 92]]

Calling the `values` attribute returns the labels along with the coordinates.

In [ ]:
b.values

(#1) [[14, 51, 71, 92, 'flower']]

`BaseBx` also exposes a method to calculate the Intersection Over Union (IOU):

In [ ]:
#| export
@patch
def iou(self:BaseBx, other):
    """Caclulates the Intersection Over Union (IOU) of the box
    w.r.t. another `BaseBx`. Returns the IOU only if the box is
    considered `valid`.
        """
    if not isinstance(other, Bx):
        other = bbx(other)
    if self.valid:
        try:
            int_box = bbx(intersection_box(self.coords, other.coords))
        except NoIntersection:
            return 0.0
        int_area = int_box.area
        union_area = other.area + self.area - int_area
        return int_area / union_area
    return 0.0 

In [ ]:
b

BaseBx(coords=[[14, 51, 71, 92]], label=['flower'])

`BaseBx` is also pseudo-iterable (calling an iterator returns `self` itself and not the coordinates or labels).

In [ ]:
#| export
@patch
def __iter__(self:BaseBx):
    """Iterates through the boxes in `BaseBx` where self.valid is True."""
    return self

@patch
def __getitem__(self:BaseBx, idx):
        """Gets the item at index idx as a BaseBx."""
        if idx>0: 
            # Fixes #2
            raise IndexError(f"BaseBx has only a single coordinate at idx=0. Got idx={idx}.")
        return self

@patch
def __next__(self:BaseBx):
    """Iteration is allowed only for valid boxes"""
    try:
        b = self[self.index]
        if not b.valid:
            # 0 area boxes are not valid
            self.index += 1
            return self.__next__()
    except IndexError:
        self.index = 0  # reset index
        raise StopIteration
    self.index += 1
    return b

In [ ]:
b = BaseBx(annots, 'flower')

In [ ]:
next(b)

BaseBx(coords=[[14, 51, 71, 92]], label=['flower'])

In [ ]:
for b_ in b:
    print(b_)

Working with multiple bounding boxes and annotaions is usually done with the help
of `MultiBx`. `MultiBx` allows iteration.

In [ ]:
# | export
class MultiBx:
    """`MultiBx` represents a collection of bounding boxes as ndarrays.
    Objects of type `MultiBx` can be indexed into, which returns a
    `BaseBx` exposing a suite of box-bound operations.
    Multiple coordinates and labels of bounding boxes can be wrapped
    as a `MultiBx` using:
        `mbx(coords, label)`.
    :param coords: can be nested coordinates of type `list` of `list`s/`json` records
        (`list`s of `dict`s)/`ndarray`s representing multiple boxes.
        If passing a list/json each index of the object should be of the following formats:
        - `list` can be formatted with `label`: `[x_min, y_min, x_max, y_max, label]`
            or without `label`: `[x_min, y_min, x_max, y_max]`
        - `dict` should be in `pascal_voc` format using the keys
            {"x_min": 0, "y_min": 0, "x_max": 1, "y_max": 1, "label": 'none'}
        If passing an `ndarray`, it should be of shape `(N,4)`.

    :param label: a `list` of `str`s that has the class name or label for the object in the
    corresponding box.
    """

    def __init__(self, coords, label: list = None):
        index = 0
        coords = coords.tolist() if isinstance(coords, np.ndarray) else list(coords)
        if label is None:
            label = [None] * len(coords)
        assert len(coords) == len(
            label
        ), f"wrong shape for coords {len(coords)} and label {len(label)}"
        bxs = [BaseBx([list(c)], l) for c, l in zip(coords, label)]
        store_attr("coords, label, index, bxs")

    def __len__(self):
        """Gets the length of coordinates."""
        return len(self.bxs)

    def __getitem__(self, idx):
        """Gets the item at index idx as a BaseBx."""
        return self.bxs[idx]

    def __iter__(self):
        """Iterates through the boxes in `MultiBx` where self.valid is True."""
        return self

    def __next__(self):
        """Iteration is allowed only for valid boxes"""
        try:
            b = self[self.index]
            if not b.valid:
                # 0 area boxes are not valid
                self.index += 1
                return self.__next__() 
        except IndexError:
            self.index = 0  # reset index
            raise StopIteration
        self.index += 1
        return b

    def __str__(self):
        return f"MultiBx(coords={self.coords}, label={self.label})"

    def __repr__(self):
        return self.__str__()

    @property
    def shape(self):
        """Returns shape of the coordinates"""
        return self.coords.shape


In [ ]:
#| export 
BX_TYPE = (Bx, MultiBx)

Generate random coordinates:

In [ ]:
np.random.seed(42)
annots = [sorted([np.random.randint(100) for i in range(4)]) for j in range(3)]
annots

[[14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99]]

All annotations are stored as a `BaseBx` in a container called `MultiBx`

In [ ]:
bxs = MultiBx(annots, ['apple', 'coke', 'tree'])
bxs

MultiBx(coords=[[14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99]], label=['apple', 'coke', 'tree'])

Each index reveals the stored coordinate as a `BaseBx`

In [ ]:
bxs[0] 

BaseBx(coords=[[14, 51, 71, 92]], label=['apple'])

They can also be iterated:

In [ ]:
next(bxs)

BaseBx(coords=[[14, 51, 71, 92]], label=['apple'])

Or using list comprehension, properties of individual boxes can be extracted

In [ ]:
[b.area for b in bxs]

[2337, 1612, 325]

In [ ]:
bxs[0].valid

True

[True, True]

In [ ]:
bxs[1].yolo()

(#1) [[51.0, 73.0, 62, 26]]

In [ ]:
bxs[0].area

2337

In [ ]:
[b_.area for b_ in bxs]

[1612, 325]

Extending `BaseBx` to also accept (`json`, `dict`) formatted coordinates and labels.

In [ ]:
#| export
class __JsonBx(MultiBx):
    """
    If five items per coordinate are passed, the last index is taken as the label.
    """

    def __init__(self, coords, label: list = None):
        super().__init__(coords, label) 

    @classmethod
    def jsonbx(cls, coords, label=None, keys=None):
        """Classmethod for JsonBx. 
        
        Also accepts keys as a list, otherwise uses `voc_keys`.
        """
        l = L()
        r = L()
        for i, c in enumerate(coords):
            assert isinstance(c, ITER_TYPES+ITER_TYPES_EXTRA), f"{inspect.stack()[0][3]} of \
                {__name__}: Expected b of type dict, got {type(c)}"
            if keys is None:
                # Fixes issue #3.
                keys = update_keys(c, default_keys=voc_keys)
            c_ = [c[k] for k in keys]  # read in order
            l_ = c_[-1] if len(c_) > 4 else "" if label is None else label[i]
            l.append(l_)
            r.append(c_[:-1] if len(c_) > 4 else c_)
        return cls(r, label=l)

    def __str__(self):
        return f'__JsonBx(coords={self.coords}, label={self.label})'

def jbx(coords=None, labels=None, keys=None):
    """Alias of the JsonBx class to process `json` records into
    `MultiBx` or `BaseBx` objects exposing many validation methods

    Also accepts keys as a list, otherwise uses `voc_keys`.
    """
    return __JsonBx.jsonbx(coords, labels, keys)

In [ ]:
annots = json.load(open('../data/annots.json'))
annots

[{'x_min': 150, 'y_min': 70, 'x_max': 270, 'y_max': 220, 'label': 'clock'},
 {'x_min': 10, 'y_min': 180, 'x_max': 115, 'y_max': 260, 'label': 'frame'}]

In [ ]:
jbx(annots, keys=voc_keys)

__JsonBx(coords=[[150, 70, 270, 220], [10, 180, 115, 260]], label=['clock', 'frame'])

Also accepts keys (for the dict) as a list, otherwise uses `voc_keys`.

In [ ]:
voc_keys

['x_min', 'y_min', 'x_max', 'y_max', 'label']

Making `MultiBx` work with lists with more than 4 items. It is a common practice
to have the class label along with the coordinates. This classmethod is useful in such 
situations

In [ ]:
ITER_TYPES

(numpy.ndarray, list, fastcore.foundation.L)

In [ ]:
#| export
class __ListBx(MultiBx):
    """
    If five items per coordinate are passed, the last index is taken as the label.
    """

    def __init__(self, coords, label: list = None):
        super().__init__(coords, label) 

    @classmethod
    def listbx(cls, coords, label=None):
        """Classmethod for __ListBx.
        """
        l = L()
        r = L()
        for i, c in enumerate(coords):
            assert isinstance(c, ITER_TYPES), f"{inspect.stack()[0][3]} of \
                {__name__}: Expected b of type list, got {type(c)}"
            l_ = label[i] if label else c[-1] if len(c) > 4 else None
            l.append(l_)
            r.append(c[:-1] if len(c) > 4 else c) 
        return cls(r, label=l)
    
    def __str__(self):
        return f'__ListBx(coords={self.coords}, label={self.label})'

def lbx(coords=None, labels=None):
    """Alias of the __ListBx class to process `list` into
    `MultiBx` or `BaseBx` objects exposing many validation methods 
    """
    return __ListBx.listbx(coords, labels)

In [ ]:
annots = [[10, 20, 100, 200, 'apple'], [40, 50, 80, 90, 'coke'], ]
annots

[[10, 20, 100, 200, 'apple'], [40, 50, 80, 90, 'coke']]

In [ ]:
lbx(annots)

__ListBx(coords=[[10, 20, 100, 200], [40, 50, 80, 90]], label=['apple', 'coke'])

In [ ]:
lbx(annots)[0]

BaseBx(coords=[[10, 20, 100, 200]], label=['apple'])

Inserting classmethod to process lists and dicts in `MultiBx`.

In [ ]:
#| export
@patch(cls_method=True)
def multibox(cls:MultiBx, coords, label: list = None, keys: list=None):
    """Classmethod for `MultiBx`. Same as mbx(coords, label).
    Calls classmethods of `JsonBx` and `ListBx` based on the type
    of coords passed.
    """
    t = explode_types(coords)
    # if explode_types returns a single class, it means they are not nested
    if t == np.ndarray: 
        return cls(coords, label)
    if t == dict: 
        b = jbx([coords], label, keys)
        return cls(b.coords, b.label)
    if t == list: 
        b = lbx([coords], label)
        return cls(b.coords, b.label)
    # if list of list or dicts
    type_l0 = list(t.keys())[0]
    type_l1 = t[type_l0][0]
    # process the data
    if type_l1 == dict: 
        b = jbx(coords, label, keys)
        return cls(b.coords, b.label)
    # process lists of lists or ndarray
    try: 
        b = lbx(coords, label)
        return cls(b.coords, b.label)
    except: 
        return cls(coords, label)        
    
def mbx(coords=None, label=None, keys=None):
    """Alias of the `MultiBx` class."""
    return MultiBx.multibox(coords, label, keys)


In [ ]:
annots_list = annots
annots_list

[[10, 20, 100, 200, 'apple'], [40, 50, 80, 90, 'coke']]

In [ ]:
annots_json = json.load(open('../data/annots.json'))
annots_json

[{'x_min': 150, 'y_min': 70, 'x_max': 270, 'y_max': 220, 'label': 'clock'},
 {'x_min': 10, 'y_min': 180, 'x_max': 115, 'y_max': 260, 'label': 'frame'}]

How the class method works:

In [ ]:
t = explode_types(annots_list)  # get all types
t

{list: [{list: [int, int, int, int, str]}, {list: [int, int, int, int, str]}]}

In [ ]:
t[list][0]  # index into the nested list and call the right class

{list: [int, int, int, int, str]}

In [ ]:
mbx(annots_json)

MultiBx(coords=[[150, 70, 270, 220], [10, 180, 115, 260]], label=['clock', 'frame'])

In [ ]:
mbx(annots_list)

MultiBx(coords=[[10, 20, 100, 200], [40, 50, 80, 90]], label=['apple', 'coke'])

In [ ]:
mbx(annots_json[0])

MultiBx(coords=[[150, 70, 270, 220]], label=['clock'])

Checking if it works with ndarrays

In [ ]:
np.random.seed(42)
annots = np.array([sorted([np.random.randint(100) for i in range(4)]) for j in range(3)])
annots

array([[14, 51, 71, 92],
       [20, 60, 82, 86],
       [74, 74, 87, 99]])

In [ ]:
mbx(annots)

MultiBx(coords=[[14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99]], label=[None, None, None])

Allowing `BaseBx` to process a single `dict` and `list` directly.

In [ ]:
#| export
@patch(cls_method=True)
def basebx(cls:BaseBx, coords, label: list = None, keys: list = voc_keys):
    """Classmethod for `BaseBx`. Same as bbx(coords, label), made to work with
    other object types other than ndarray."""
    try:
        coords, label = make_single_iterable(coords, keys=keys)
    except ValueError:
        coords = make_single_iterable(coords)
    finally:
        return cls(coords, label)

def bbx(coords=None, labels=None, keys=voc_keys):
    """Alias of the `BaseBx` class."""
    return BaseBx.basebx(coords, labels, keys)

Remember that `BaseBx` can only have one box coordinate and label at a time.

In [ ]:
annots_list

[[10, 20, 100, 200, 'apple'], [40, 50, 80, 90, 'coke']]

What does `make_single_iterable` do? It converts a single list or dict of 
coordinates into an iterable list that can be used by `BaseBx`.

In [ ]:
annots_list[0]

[10, 20, 100, 200, 'apple']

In [ ]:
make_single_iterable(annots_list[0], keys=voc_keys)

((#1) [[10, 20, 100, 200]], ['apple'])

The class method makes it easier to directly call `BaseBx` without making the coordinates a list of list.

In [ ]:
bbx(annots_list[0])

BaseBx(coords=[[10, 20, 100, 200]], label=['apple'])

In [ ]:
annots_list[0][:-1]

[10, 20, 100, 200]

In [ ]:
bbx(annots_list[0][:-1])  # if label is not passed

BaseBx(coords=[[10, 20, 100, 200]], label=[])

In [ ]:
bbx(annots_json[0])

BaseBx(coords=[[150, 70, 270, 220]], label=['clock'])

# `get_bx`

When in doubt, use `get_bx`.

In [ ]:
ITER_TYPES

(numpy.ndarray, list, fastcore.foundation.L)

In [ ]:
#| export 

def get_bx(coords, label=None):
    """
    Helper function to check and call the correct type of Bx instance.

    Checks for the type of data passed and calls the respective class 
    to generate a Bx instance. Currently only supports ndarray, list, dict, 
    tuple, nested list, nested tuple.

    Parameters
    ----------
    coords : ndarray, list, dict, tuple, nested list, nested tuple
        Coordinates of anchor boxes.
    label : list, optional
        Labels for anchor boxes in order, by default None

    Returns
    -------
    Bx
        An instance of MultiBx, ListBx, BaseBx or JsonBx

    Raises
    ------
    NotImplementedError
        If unknown type of coordinates are passed.
    """
    # process ndarray
    if isinstance(coords, np.ndarray):
        coords = np.atleast_2d(coords)
        return mbx(coords, label)
    # process list
    if isinstance(coords, (list, L)): 
        if isinstance(coords[0], COORD_TYPES):
            """If first item is a position"""
            return bbx(coords, label)
        elif isinstance(coords[0], ITER_TYPES+ITER_TYPES_EXTRA):
            """If fist item is an iterable"""
            return mbx(coords, label)
        elif isinstance(coords[0], ITER_TYPES_TUPLE):
            """If first item is a tuple"""
            return mbx([list(c) for c in coords], label)
    # process dict
    if isinstance(coords, dict):
        return bbx([coords], label)
    # process tuple
    if isinstance(coords, tuple):
        return bbx(list(coords), label)
    # process BX_TYPE
    if isinstance(coords, BX_TYPE):
        return coords
    else:
        raise NotImplementedError(
            f"{inspect.stack()[0][3]} of {__name__}: Got coords={coords} of type {type(coords)}."
        )

`get_bx` runs a bunch of if-else statements to call the right module when in doubt.

In [ ]:
annots_json

[{'x_min': 150, 'y_min': 70, 'x_max': 270, 'y_max': 220, 'label': 'clock'},
 {'x_min': 10, 'y_min': 180, 'x_max': 115, 'y_max': 260, 'label': 'frame'}]

In [ ]:
get_bx(annots_json)

MultiBx(coords=[[150, 70, 270, 220], [10, 180, 115, 260]], label=['clock', 'frame'])

In [ ]:
len(annots_json[0])

5

In [ ]:
get_bx([annots_json[0]])

MultiBx(coords=[[150, 70, 270, 220]], label=['clock'])

In [ ]:
get_bx(annots_list)

MultiBx(coords=[[10, 20, 100, 200], [40, 50, 80, 90]], label=['apple', 'coke'])

In [ ]:
get_bx([0, 1, 0, 1])

BaseBx(coords=[[0, 1, 0, 1]], label=[])

Enabling stacking of different boxes.

In [ ]:
#| export
@patch
def __add__(self:BaseBx, other):
        """Pseudo-add method that stacks the provided boxes and labels. Stacking two
        boxes imply that the resulting box is a `MultiBx`: `BaseBx` + `BaseBx`
        = `MultiBx`. This violates the idea of `BaseBx` since the result
        holds more than 1 coordinate/label for the box.
        From `v.2.0`, a `UserWarning` is issued if called.
        Recommended use is either: `BaseBx` + `BaseBx` = `MultiBx` or
        `basics.stack_bxs()`.
        """
        if not isinstance(other, BX_TYPE):
            raise TypeError(
                f"{inspect.stack()[0][3]} of {__name__}: Expected a subclass of {BX_TYPE}"
            )
        else:
            warnings.warn(
                BxViolation(
                    f"Change of object type imminent if trying to add "
                    f"{type(self)}+{type(other)}. Use {type(other)}+{type(self)} "
                    f"instead or basics.stack_bxs()."
                )
            )
        coords = self.coords + other.coords
        label = self.label + other.label
        return mbx(coords, label)

@patch
def __add__(self:MultiBx, other):
    """Pseudo-add method that stacks the provided boxes and labels. Stacking two
    boxes imply that the resulting box is a `MultiBx`: `MultiBx` + `MultiBx`
    = `MultiBx`. Same as `basics.stack_bxs()`.
    """
    if not isinstance(other, BX_TYPE):
        raise TypeError(
            f"{inspect.stack()[0][3]} of {__name__}: Expected type {BX_TYPE}, "
            f"got self={type(self)}, other={type(other)}"
        )
    coords = self.coords + other.coords
    label = self.label + other.label
    return mbx(coords, label)

In [ ]:
#| export
def stack_bxs(b1, b2):
    """
    Method to stack two Bx-types together. Similar to `__add__` of BxTypes
    but avoids UserWarning.
    :param b1: 
    :param b2: 
    :return:  
    _summary_

    Parameters
    ----------
    b1 : Bx, MultiBx
        Anchor box coordinates Bx
    b2 : Bx, MultiBx
        Anchor box coordinates Bx

    Returns
    -------
    MultiBx
        Stacked anchor box coordinates of MultiBx type.

    Raises
    ------
    TypeError
        If unknown type of coordinates are passed.
    """

    if not isinstance(b1, BX_TYPE):
        raise TypeError(
            f"{inspect.stack()[0][3]} of {__name__}: Expected type {BX_TYPE}, got b1={type(b1)}"
        )
    if not isinstance(b2, BX_TYPE):
        raise TypeError(
            f"{inspect.stack()[0][3]} of {__name__}: Expected type {BX_TYPE}, got b2={type(b2)}"
        )
    if isinstance(b1, BaseBx):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            return b1 + b2
    return b1 + b2


def add_bxs(b1, b2):
    """Alias of stack_bxs()."""
    return stack_bxs(b1, b2)
 

In [ ]:
b

BaseBx(coords=[[14, 51, 71, 92]], label=['flower'])

Internally this is what is done to stack them:

In [ ]:
bxs.coords + b.coords, bxs.label + b.label

([[14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99], [14, 51, 71, 92]],
 (#4) ['apple','coke','tree','flower'])

In [ ]:
bxs + b

MultiBx(coords=[[14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99], [14, 51, 71, 92]], label=['apple', 'coke', 'tree', 'flower'])

Adding a `MultiBx` to a `BaseBx` makes the new set of coordinates a `MultiBx`, so a `BxViolation` warning is issued
if this was not intended. 

In [ ]:
b + bxs

/home/gg/data/pybx/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:19: BxViolation: Change of object type imminent if trying to add <class '__main__.BaseBx'>+<class '__main__.MultiBx'>. Use <class '__main__.MultiBx'>+<class '__main__.BaseBx'> instead or basics.stack_bxs().


MultiBx(coords=[[14, 51, 71, 92], [14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99]], label=['flower', 'apple', 'coke', 'tree'])

In [ ]:
stack_bxs(b, bxs)

MultiBx(coords=[[14, 51, 71, 92], [14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99]], label=['flower', 'apple', 'coke', 'tree'])

To avoid the `BxViolation`, use the method `stack_bxs`.

In [ ]:
stack_bxs(bxs, b)

MultiBx(coords=[[14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99], [14, 51, 71, 92]], label=['apple', 'coke', 'tree', 'flower'])